In [1]:
# all imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from threading import Timer
import csv

In [2]:
url = 'https://www.airbnb.com/s/Chicago--IL--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=january&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&adults=2&source=structured_search_input_header&search_type=user_map_move&room_types%5B%5D=Entire%20home%2Fapt&ne_lat=41.98964427255951&ne_lng=-87.6077828578949&sw_lat=41.84249090396328&sw_lng=-87.75946714210511&zoom=12&search_by_map=true&place_id=ChIJ7cv00DwsDogRAMDACa2m4K8&federated_search_session_id=c74980a1-e7f4-46f4-aef2-59e1e842ccf7&pagination_search=true'
allplaces = []
page_count = 0
options = Options()
options.add_argument('--blink-settings=imagesEnabled=false')
driver = webdriver.Chrome(options=options)
driver.set_window_position(0, 0)
driver.set_window_size(2560, 1440)


In [3]:
def dict_to_csv():

    field_names = ['title','location','rating','reviews','guests','bedrooms','beds','bathrooms','price','days booked']

    with open('airbnb.csv', 'w',encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = field_names)
        writer.writeheader()
        writer.writerows(allplaces)



In [4]:
def pageload_checker():
    try:
        a = {
        'title':driver.find_element_by_class_name('_fecoyn4').text,
        'guests':driver.find_element_by_class_name('_tqmy57').find_elements_by_xpath('./div[2]/span[1]')[0].text[0],
        'bedrooms':driver.find_element_by_class_name('_tqmy57').find_elements_by_xpath('./div[2]/span[4]')[0].text[0],
        'beds':driver.find_element_by_class_name('_tqmy57').find_elements_by_xpath('./div[2]/span[7]')[0].text[0],
        'bathrooms':driver.find_element_by_class_name('_tqmy57').find_elements_by_xpath('./div[2]/span[10]')[0].text[0],
        'price':driver.find_element_by_class_name('_1n700sq').get_attribute('innerText'),
        'days booked':0
        }
        b = driver.find_element_by_class_name('_1hk9h0u')
    except:
        return False
    return True

In [5]:
def get_info_from_placepage(index,links,locations):
    try:
        rating = driver.find_element_by_class_name('_1ne5r4rt').text
    except:
        rating = ''
    try:
        reviews = driver.find_element_by_class_name('_142pbzop').text
    except:
        reviews = ''
    
    place_dict = {
        'title':driver.find_element_by_class_name('_fecoyn4').text,
        'location':locations[index][7:],
        'rating':rating,
        'reviews':reviews[1:reviews.find('r')],
        'guests':driver.find_element_by_class_name('_tqmy57').find_elements_by_xpath('./div[2]/span[1]')[0].text[0],
        'bedrooms':driver.find_element_by_class_name('_tqmy57').find_elements_by_xpath('./div[2]/span[4]')[0].text[0],
        'beds':driver.find_element_by_class_name('_tqmy57').find_elements_by_xpath('./div[2]/span[7]')[0].text[0],
        'bathrooms':driver.find_element_by_class_name('_tqmy57').find_elements_by_xpath('./div[2]/span[10]')[0].text[0],
        'price':driver.find_element_by_class_name('_1n700sq').get_attribute('innerText')[1:driver.find_element_by_class_name('_1n700sq').get_attribute('innerText').find('/')-1],
        'days booked':0
    }
    driver.find_element_by_class_name('_1hk9h0u').click()
    count = 0
    calendar = driver.find_elements_by_tag_name('td')
    for date in calendar:
        if not date.get_attribute('aria-label') == None:
            if 'Not available' in date.get_attribute('aria-label'):
                count = count +1
    place_dict['days booked'] = count
    allplaces.append(place_dict)
    global url
    if index == len(links)-1:
        if url == None:
            dict_to_csv()
            driver.quit
            print('done')
            return
        next_page()
        return
    driver.get(links[index+1])
    while True:
        if pageload_checker():
            Timer(1.0, get_info_from_placepage,[index+1,links,locations]).start()
            break
        try:
            if driver.find_element_by_class_name('_1uippt2').text[0] == 'P':
                get_info_from_placepage(1,links,locations)
        except:
            pass

In [6]:

def listing_scan():
    links = driver.find_elements_by_class_name('_mm360j')
    locations = driver.find_elements_by_class_name('_1xzimiid')
    for i in range(len(locations)):
        links[i] = links[i].get_attribute('href')
        locations[i] = locations[i].text
    
    global url
    url = driver.find_element_by_class_name('_za9j7e').get_attribute('href') 
    print(0)
    driver.get(links[0])
    while True:
        if pageload_checker():
            Timer(0.0, get_info_from_placepage,[0,links,locations]).start()
            break
        try:
            if driver.find_element_by_class_name('_1uippt2').text[0] == 'P':
                get_info_from_placepage(1,links,locations)
                break
        except:
            pass


In [7]:
def next_page():
    driver.get(url)
    Timer(6.0,listing_scan,[]).start()

    

In [8]:
next_page()

In [9]:
#driver.quit


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
done
